In [ ]:
import numpy as np 
import os 
from PIL import Image 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 
from random import randint
from sklearn.model_selection import train_test_split
from keras import layers
from keras import models
import tensorflow as tf
 

# **Loading the Dataset and Preprocessing**

In [ ]:
lookup = dict()
reverselookup = dict()
count = 0
for j in os.listdir('archive/leapgestrecog/leapGestRecog/00/'):
    if not j.startswith('.'): 
        lookup[j] = count
        reverselookup[count] = j
        count = count + 1
lookup

In [ ]:
x_data = []
y_data = []
datacount = 0 # 
for i in range(0, 10): 
    for j in os.listdir('archive/leapgestrecog/leapGestRecog/0' + str(i) + '/'):
        if not j.startswith('.'): 
            count = 0 
            for k in os.listdir('archive/leapgestrecog/leapGestRecog/0' + 
                                str(i) + '/' + j + '/'):
                img = Image.open('archive/leapgestrecog/leapGestRecog/0' + 
                                 str(i) + '/' + j + '/' + k).convert('L')
                img = img.resize((320, 120))
                arr = np.array(img)
                x_data.append(arr) 
                count = count + 1
            y_values = np.full((count, 1), lookup[j]) 
            y_data.append(y_values)
            datacount = datacount + count
x_data = np.array(x_data, dtype = 'float32')
y_data = np.array(y_data)
y_data = y_data.reshape(datacount, 1)

In [ ]:
for i in range(0, 10):
    plt.imshow(x_data[i*200 , :, :])
    plt.title(reverselookup[y_data[i*200 ,0]])
    plt.show()

In [ ]:
import keras
from keras.utils import to_categorical
y_data = to_categorical(y_data)

In [ ]:
x_data = x_data.reshape((datacount, 120, 320, 1))
x_data /= 255

In [ ]:
x_train,x_further,y_train,y_further = train_test_split(x_data,y_data,test_size = 0.2)
x_validate,x_test,y_validate,y_test = train_test_split(x_further,y_further,test_size = 0.5)

# Defining the Model

**Base Model**

In [ ]:
model=models.Sequential()
model.add(layers.Conv2D(32, (5, 5), strides=(2, 2), activation='relu', input_shape=(120, 320,1),name = 'conv1')) 
model.add(layers.MaxPooling2D((2, 2),name = 'pool1'))
model.add(layers.Conv2D(64, (3, 3), activation='relu',name = 'conv2')) 
model.add(layers.MaxPooling2D((2, 2),name = 'pool2'))
model.add(layers.Conv2D(64, (3, 3), activation='relu',name = 'conv3'))
model.add(layers.MaxPooling2D((2, 2),name = 'pool3'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu',name = 'dense1'))
model.add(layers.Dense(10, activation='softmax', name ='output'))
model.summary()

**Updated Model**

In [ ]:
model=models.Sequential()
model.add(layers.Conv2D(22, (5, 5), strides=(2, 2), activation='relu', input_shape=(120, 320,1) , name='conv1')) 
model.add(layers.MaxPooling2D((2, 2), name='pool1'))
model.add(layers.Conv2D(18, (2, 2), activation='relu', name='conv2')) 
model.add(layers.MaxPooling2D((2, 2), name='pool2'))
model.add(layers.Conv2D(16, (3, 3), activation='relu', name='conv3'))
model.add(layers.MaxPooling2D((2, 2),name ='pool3'))
model.add(layers.Flatten())
model.add(layers.Dense(2, activation='relu',name ='dense1'))
model.add(layers.Dense(8, activation='relu', name = 'dense2'))
model.add(layers.Dense(6, activation='relu' , name = 'dense3'))
model.add(layers.Dense(10, activation='softmax',name = 'output'))
model.summary()

**Training the Model**

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=64, verbose=1, validation_data=(x_validate, y_validate))

In [ ]:
[loss, acc] = model.evaluate(x_test,y_test,verbose=1)

Saving the original weights of model

In [ ]:
original_w = model.get_weights()

Saving the model as h5 file.

In [ ]:
model.save('MyModel',save_format='h5')

# **Changing the weights of model layer by layer**

In [ ]:
num_layers=3
accuracys=[]
x=num_layers
for i in range(len(original_w)-2,len(original_w)-1-2*num_layers,-2):
  accuracy=[]
  print("Layer :",x)
  for scalar in [0.001,0.01,0.1,.5, .75,1.5,1.75,2.,4.]:
    new_w=[]
    for w in range(len(original_w)):
      if(w==i):
        new_w.append(original_w[w]+scalar)
      else:
        new_w.append(original_w[w])
    model.set_weights(new_w)
    evaluation = model.evaluate(x_test, y_test)
    print("Scallar {} --> test accuracy {}".format(scalar,evaluation[1]))
    accuracy.append(evaluation[1])
    model.set_weights(original_w)
  x-=1
  model.set_weights(original_w)
  accuracys.append(accuracy)

**Plotting the graph of accuracy vs Scalars**

In [ ]:
scalars= [0.001,0.01,0.1,.5, .75,1.5,1.75,2.,4.]
for i in range(len(accuracys)):
  plt.plot(scalars, accuracys[i], label=f'layer {num_layers-i}')
plt.xlabel('Scalar')
plt.ylabel('Accuracy')
plt.title('Effect on accuracy when weights are added by scalers')
plt.legend()
plt.show()

Average accuracy of model when weights are altered in different layers.

In [ ]:
average=[]
for i in accuracys:
  average.append(sum(i)/len(i))
print(average)

# Changing all the weights of model simultaneously.

In [ ]:
scalars=[]
accuracys=[]
for scalar in [0.00001,0.0001,0.001, 0.05, 0.1, .25, 0.5, .75, 1., 1.1,1.5,2,3,4]:
    new_w = [scalar*w for w in original_w]
    model.set_weights(new_w)
    evaluation = model.evaluate(x_test, y_test)
    print("Scallar {} --> test accuracy {}".format(scalar,evaluation[1]))
    scalars.append(scalar)
    accuracys.append(evaluation[1])

**Ploting the graph between accuracy and scalars used.**

In [ ]:
scalars=[]
accuracys=[]
plt.plot(scalars, accuracys)
plt.xlabel('Scalar')
plt.ylabel('Accuracy')
plt.title('Effect on accuracy when weights are multiplied by scalers')
plt.show()

# Optimizating the model by Quantization

In [ ]:
model.summary()

# HLS FOR ML CODE 

In [ ]:
x_test_normalized = x_test / 255

In [ ]:
# import hls4ml

# # config = hls4ml.utils.config_from_keras_model(model, granularity='model')
# # print("-----------------------------------")
# # print("Configuration")
# # # plotting.print_dict(config)
# # print("-----------------------------------")
# # hls_model = hls4ml.converters.convert_from_keras_model(
# #     model, hls_config=config, output_dir='model_1/hls4ml_prj', part='xczu7ev-ffvc1156-2-e'
# # )

# config = hls4ml.utils.config_from_keras_model(model, granularity='name')
# config['Model']['ReuseFactor'] = 1
# config['LayerName']['conv1']['Strategy'] = 'Latency'
# config['LayerName']['conv2']['Strategy'] = 'Latency'
# config['LayerName']['pool1']['Strategy'] = 'Latency'
# config['LayerName']['pool2']['Strategy'] = 'Latency'
# config['LayerName']['pool3']['Strategy'] = 'Latency'
# config['LayerName']['conv3']['Strategy'] = 'Latency'
# config['Model']['Precision'] = 'ap_fixed<16,6>'
# config['LayerName']['output']['Strategy'] = 'Stable'
# config['IOType'] = 'io_stream'  # Must set this if using CNNs!
# #config['HLSConfig'] = config
# #config['KerasModel'] = model
# #config['OutputDir'] = 'normal_cnn/'

# for layer in config['LayerName']:
#    # config['LayerName'][layer]['Trace'] = True
#    config['LayerName'][layer]['ReuseFactor'] = 64
# print("-----------------------------------")
# #plotting.print_dict(config)
# print("-----------------------------------")



# hls_model = hls4ml.converters.convert_from_keras_model(model,
#                                                        hls_config=config,
#                                                        output_dir='model_3/hls4ml_prj_pynq',
#                                                        backend='VivadoAccelerator',
#                                                        part='xczu7ev-ffvc1156-2-e')



import hls4ml

# config = hls4ml.utils.config_from_keras_model(model, granularity='model')
# print("-----------------------------------")
# print("Configuration")
# # plotting.print_dict(config)
# print("-----------------------------------")
# hls_model = hls4ml.converters.convert_from_keras_model(
#     model, hls_config=config, output_dir='model_1/hls4ml_prj', part='xczu7ev-ffvc1156-2-e'
# )


hls_config = hls4ml.utils.config_from_keras_model(model, granularity='name')
hls_config['Model']['ReuseFactor'] = 2584
hls_config['LayerName']['conv1']['Strategy'] = 'Latency'
hls_config['LayerName']['conv2']['Strategy'] = 'Latency'

hls_config['LayerName']['pool1']['Strategy'] = 'Latency'


hls_config['Model']['Precision'] = 'ap_fixed<8,2>'
hls_config['LayerName']['output']['Strategy'] = 'Stable'
# #plotting.print_dict(hls_config_aq) 
 
cfg = hls4ml.converters.create_config(backend='VivadoAccelerator')
cfg['IOType'] = 'io_stream'  # Must set this if using CNNs!
cfg['HLSConfig'] = hls_config
cfg['KerasModel'] = model
cfg['OutputDir'] = 'normal_cnn/'
cfg['Part'] = 'xczu7ev-ffvc1156-2-e'

hls_model = hls4ml.converters.keras_to_hls(cfg)



hls_config = hls4ml.utils.config_from_keras_model(model, granularity='model')
hls_config['Flows'] = ['vivadoaccelerator:fifo_depth_optimization']
print("-----------------------------------")

hls4ml.model.optimizer.get_optimizer('vivado:fifo_depth_optimization').configure(profiling_fifo_depth=100_000)
 

print("----------------compilation starts-------------------")
hls_model.compile()
# X_test = np.ascontiguousarray(x_test_normalized)
# y_hls = hls_model.predict(X_test)
print("----------------compilation done-------------------")

 


In [ ]:
hls_model.build(csim=False, synth=True, vsynth=True)

In [ ]:
hls4ml.report.read_vivado_report( 'normal_cnn/hls4ml_prj')